______
<img style="float: right;" src="./images/headerlogo.png">
## Google Earth Engine Tutorial
### Basic visualisation
_____

To visualize an image, the following options, related to the image, can be specified:
    
+ *bands*: 	List of three band names to be display in the RGB channels. This is relevant for multiband images.<br> 
+ *palette*:	Color palette in a list of hexadecimal code strings. Apply to single-band images.<br> 
+ *opacity*: 	The transparency of the layer (0.0 is fully transparent and 1.0 is fully opaque).<br> 

Besides, the following visualisation parameters can be specified: <br> 
+ *min*:	Value(s) to map to 0. <br> 
+ *max*:  	Value(s) to map to 255. <br>
+ *gain*:  	Value(s) by which to multiply each pixel value.<br> 
+ *bias*:  	Value(s) to add to each pixel value.<br> 
+ *gamma*:  	Gamma correction factor(s).<br> 

The above parameters are applied per image band. When a multiband image is being displayed, using a single number means that it will be applied to the three bands. Otherwise, a list of three numbers, one for each band can be set.

In GEE playground,  an image can be displayed using the function *Map.addLayer* with parameters listed above. In this tutorial we will use the function *getThumbUrl* that generates a thumbnail url of the image preview. This preview will be open using *Image* from the package Ipython.display. 

Parameters for the thumbnail can be specified to generate the image. For example, it can be specified bands to display and additional parameters as follows: <br>
+ *dimensions:* a pair of number specifying the maximum *Width*x*Height* dimensions of the thumbnail. If only one value is given, it is used as the maximum and the other dimension is computed proportionally. <br>
+ *region:* region of the image to produce the thumbnail given as GeoJSON or coordinates.<br> 
+ *format:* format of the output given as a string ('png' or 'jpg'). <br> 

In the following code we will visualize an image using *getThumbUrl*, later on we will apply the *folium* library. 

In [1]:
# import ee python package
import ee 
ee.Initialize() 
# import Ipython.display for visualisation
from IPython.display import Image  

In [2]:
# load the image 
image = ee.Image("users/rosamaguilar/tutorial/subset")
# the maximun pixel value is 2048 because the image has 11-bit per pixel but we are 
# making an 8-bit display image, then the pixel values need to be stretch to fit in the range (0,255).   
url  = image.getThumbUrl({'min':0, 'max':2048})
# you could copy and paste this url in a browser to see the image 
print(url)
# display image thumbnails.
Image(url=url)

https://earthengine.googleapis.com/api/thumb?thumbid=f8d71d599bc69699cf15fea443c10690&token=eac5fc716765cae9a7d144aea1753c62


In [3]:
# We can specify the bands to make a RGB composite.
# First, get the band names of the image
names = image.bandNames();
print(names.getInfo())

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8']


In [4]:
# As you may observe the image has 8 bands because is a World-view2 image.
# We may want to select only the band in the visible range, this mean bands blue, green and red (bands: b2, b3,b5)  
# To display a RGB natural color composite, the band order must be set to [b5,b3,b2] 
# gamma corrections factors (one per band)
Image(url=image.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b5,b3,b2', 'gamma': '0.95, 1.1, 1'}))

In [5]:
# Apply "false color" composite
Image(url=image.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b7,b5,b3', 'gamma': '1, 1, 1'}))

You may try other band combinations and gamma factors.

In [6]:
# We may want to create an image with a selected of bands. Also, we can rename band names 
# as can be seen as follows:
#
visimage = image.select(   
   ['b5','b3','b2'],   # current band names
    ['Red', 'Green', 'Blue']  # new band names
)
# print the new names 
print(visimage.bandNames().getInfo()) 

['Red', 'Green', 'Blue']


#### Visualization using Folium

The function *getThumbUrl* is useful to display a preview of an image. But it is not possible to have any kind of interaction (zoom in, zoom out, etc). Besides, there is not an equivalent function to display vector layers (feature collections). To perform this tasks, we will used the [folium library](https://github.com/python-visualization/folium).

A function based on folium to interact with GEE are available in https://github.com/python-visualization/folium_contrib/blob/master/GoogleEarthEngine_layer.ipynb

For more information about folium, visit: https://github.com/python-visualization/folium


In [7]:
# import GEE and foliumn
import ee
import folium
ee.Initialize()

In [8]:
# define a function to interact with GEE
# from https://github.com/python-visualization/folium_contrib/blob/master/GoogleEarthEngine_layer.ipynb
def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """ Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    print(folium_kwargs['overlay'])
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)


In a previous step we upload an image to a GEE asset and a vector layer to a google drive as a feature table. 
We will visualize this layer using folium. First, the feature collection is retrieved in a GeoJson format. Later, it will be display in the folium map. 


In [9]:
# load the fusion table into a feature collection
fc = ee.FeatureCollection ('ft:1K41m-umQ1K8Ys-9bVTqoEYzwKD6xEzIVTPELdVkK')
geojs = fc.getDownloadURL("json")

# retrieve the fc as a json to display in the map
import urllib
with urllib.request.urlopen(geojs) as url:
    fcjs = url.read()
fcjs = str(fcjs, 'utf-8')  


In [10]:
# create a folium map
m1 = folium.Map([12.1724,-5.1874], zoom_start=14)
# draw the vector layer 
folium.GeoJson(
    fcjs,
    name='Parcels',
    style_function=lambda feature: {
        'fillColor': '#ff0000',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    }
).add_to(m1)

# load the image
image = ee.Image("users/rosamaguilar/tutorial/subset")  

# specify visualization parameters
vis_params = {'min': 0, 'max':2048, 'bands':'b5,b3,b2'}

# add the image "subset" using the function previoulsy defined
folium_gee_layer(m1,image,vis_params,folium_kwargs={'overlay':True,'name':'Subset'})

# add a control layer to the map
m1.add_child(folium.LayerControl())

# shows the map
m1

True
